In [2]:
import lasio
import numpy as np
import os
import pandas as pd

# Create data directory
output_dir = "../data"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print(f"Generating 10 synthetic LAS files in {output_dir}...")

# Simulation parameters
n_samples = 500
depth = np.linspace(1000, 1500, n_samples)

# Variations to simulate real-world messiness
mnemonic_variations = [
    {'GR': 'GR', 'NPHI': 'NPHI', 'RHOB': 'RHOB'},
    {'GR': 'GAMMA', 'NPHI': 'NEUT', 'RHOB': 'DENS'},
    {'GR': 'G.R.', 'NPHI': 'NPHI_V1', 'RHOB': 'RHOB_FINAL'},
    {'GR': 'GR', 'NPHI': 'NPHI', 'RHOB': 'RHOB'}, # Standard
    {'GR': 'GAMMA_RAY', 'NPHI': 'NPHI', 'RHOB': 'RHOB'},
    {'GR': 'GR', 'NPHI': 'NPHI', 'RHOB': 'RHOB'},
    {'GR': 'GR', 'NPHI': 'NPHI', 'RHOB': 'RHOB'},
    {'GR': 'G.R.', 'NPHI': 'NPHI', 'RHOB': 'RHOB'},
    {'GR': 'GR', 'NPHI': 'NEUT', 'RHOB': 'DENS'},
    {'GR': 'GAMMA', 'NPHI': 'NPHI', 'RHOB': 'RHOB'}
]

for i in range(10):
    las = lasio.LASFile()
    
    # Fill Header
    las.well.WELL = f"WELL-{i+1:03d}"
    las.well.NULL = -999.25  # Standard null for LAS
    
    # Generate Synthetic Curves with some noise/nulls
    # Gamma Ray (0-150)
    gr_data = np.random.normal(60, 15, n_samples) + np.sin(depth/50)*20
    
    # Neutron Porosity (0.1 - 0.45)
    nphi_data = np.random.normal(0.25, 0.05, n_samples)
    
    # Density (1.95 - 2.65)
    rhob_data = 2.65 - (nphi_data * 1.2) + np.random.normal(0, 0.02, n_samples)

    # Injecting intentional Nulls (messy data)
    if i % 3 == 0:
        gr_data[50:100] = -999.25 # Block of nulls
    if i % 4 == 0:
        nphi_data[200:250] = -999.25
        
    # Add curves with varied mnemonics
    mnemonics = mnemonic_variations[i]
    
    # Use append_curve instead of add_curve
    las.append_curve('DEPT', depth, unit='M')
    las.append_curve(mnemonics['GR'], gr_data, unit='API', descr='Gamma Ray')
    las.append_curve(mnemonics['NPHI'], nphi_data, unit='V/V', descr='Neutron Porosity')
    las.append_curve(mnemonics['RHOB'], rhob_data, unit='G/C3', descr='Bulk Density')
    
    # Write file
    filename = os.path.join(output_dir, f"well_{i+1:03d}.las")
    las.write(filename)

print("Generation Complete.")

Generating 10 synthetic LAS files in ../data...
Generation Complete.
